实现有向图的 SCC 求解

应用背景：

**ATP: Directed Graph Embedding with Asymmetric Transitivity Preservation**

ATP代码[地址](https://github.com/zhenv5/atp)

[paperswithcode](https://paperswithcode.com/paper/atp-directed-graph-embedding-with-asymmetric)

论文地址：https://arxiv.org/pdf/1811.00839v2.pdf

Abstract: Directed graphs have been widely used in Community Question Answering services (CQAs) to model asymmetric relationships among different types of nodes in CQA graphs, e.g., question, answer, user....

数据集 wiki-vote

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
dataset_path = os.path.abspath('.') + r'/dataset'
print(dataset_path)

/home/yuchen/VSProjects/test/SCC/dataset


In [4]:
raw_data = pd.read_csv(dataset_path + r'/Wiki-Vote.tsv', sep='\t')
print(raw_data.iloc[0:10,:])

   FromNodeId  ToNodeId
0          30      1412
1          30      3352
2          30      5254
3          30      5543
4          30      7478
5           3        28
6           3        30
7           3        39
8           3        54
9           3       108


In [16]:
data = raw_data.to_numpy()
print(data.shape)
print(data[:10])
np.savetxt(dataset_path + r'/Wiki-Vote.txt', data, delimiter=' ', fmt='%d')

(103689, 2)
[[  30 1412]
 [  30 3352]
 [  30 5254]
 [  30 5543]
 [  30 7478]
 [   3   28]
 [   3   30]
 [   3   39]
 [   3   54]
 [   3  108]]


In [17]:
unique_data = np.unique(data)
print(unique_data.shape)
np.sort(unique_data)
index = [x for x in range(0,len(unique_data))]
dict_node = dict(zip(unique_data, index))
reverse_dict = dict(zip(index, unique))
new_id = [x for x in range(0,7115)]

(7115,)
[ 3  4  5  6  7  8  9 10 11 12]


In [6]:
# 使用 dgl 构建图，并用 networkx 可视化
import torch
import dgl
import networkx as nx
import matplotlib.pyplot as plt


Using backend: pytorch


In [7]:
u = data[:,0].reshape(data.shape[0])
v = data[:,1].reshape(data.shape[0])
print(u.shape,v.shape)
print(u[:10],v[:10])
# 查看总结点数
s = pd.unique(raw_data[['FromNodeId', 'ToNodeId']].values.ravel())
print(len(s))

(103689,) (103689,)
[30 30 30 30 30  3  3  3  3  3] [1412 3352 5254 5543 7478   28   30   39   54  108]
7115


In [8]:
# 此处求解
g = dgl.graph((u, v))
print(g)
print(g.nodes())
print(g.edges())
print(g.ntypes)

Graph(num_nodes=8298, num_edges=103689,
      ndata_schemes={}
      edata_schemes={})
tensor([   0,    1,    2,  ..., 8295, 8296, 8297])
(tensor([  30,   30,   30,  ..., 8150, 8150, 8274]), tensor([1412, 3352, 5254,  ..., 8275, 8276, 8275]))
['_N']


In [9]:
# nx.draw(g.to_networkx(), with_labels=False)
# plt.savefig('wiki-vote')

In [10]:
def create_adjmat(data):
    M = np.zeros((8298,8298)) # 这个数据集未经过处理有8298点，实际只有7115点
    print(data.shape)
    for u,v in zip(data[:,0],data[:,1]):
        M[u][v] = 1
    np.savetxt(dataset_path + r'/Wiki-Vote.txt', M, delimiter=' ')

        
df = pd.read_csv(dataset_path + r'/Wiki-Vote.tsv', sep='\t')
df = df.to_numpy()
create_adjmat(df)

(103689, 2)


In [ ]:
# 转化为邻接矩阵, 使用tarjan来实现强连通分量的求解
# 首先转化label encode之后的图

